# Imports

In [2]:
import tensorflow as tf
import tensorflow_text as tf_text
import random
from collections import Counter
import numpy as np
import tqdm as tqdm
import time
import re
from tensorflow.keras.layers import Layer, Embedding, Softmax

# Open File


In [3]:
with open('bible.txt', 'r') as file:
    bible = file.read().replace('\n', ' ')

## Preprocess

In [4]:
def prep(text,k = 10000, splitter = "whitespace", with_dot = False):
    p = text.lower()
    
    if(with_dot):
        p = re.sub('[^A-Za-z. ]+', '', p)
    else: 
        p = re.sub('[^A-Za-z ]+', '', p)
    
    split_it = p.split()
    counter = Counter(split_it)
    most_occur = counter.most_common(k)

    p = ' '.join(p.split())

    if splitter == "whitespace":
        splitter = tf_text.WhitespaceTokenizer()
        p = splitter.split(p)  


    return p, split_it, most_occur


In [5]:
pbible, ogbible, most_occur = prep(bible)

# Splitter

In [6]:
print(pbible)
#print(type(ogbible))

tf.Tensor([b'the' b'first' b'book' ... b'you' b'all' b'amen'], shape=(790017,), dtype=string)


In [7]:
def indexer(word, wlist):
    matched_indeces = []
    i = 0
    length = len(wlist)
    while i < length:
        if word == wlist[i]:
            matched_indeces.append(i)
        i += 1
    return matched_indeces

In [8]:
#print(indexer('the', ogbible))

In [9]:
def find_context(word, bible, k = 4):
    scope = []
    indexes = indexer(word, bible)
    index = indexes[random.randint(0, len(indexes)-1)]
    for i in range(int(k/2)):
        scope.append((word,bible[index-(i+1)]))
        scope.append((word,bible[index+(i+1)]))
    return scope

find_context() needs to work with most_occur

In [27]:
#print(find_context('jesus', ogbible)[0])
print(find_context('jesus', most_occur)[0])


('jesus', 'and')


In [30]:
#print(most_occur[i][0])
print(ogbible[:5])

['the', 'first', 'book', 'of', 'moses']


In [33]:
def create_dataset(bible, k = 10000):
    
    pairs = []
    for word in bible:
        pairs.append(find_context(word, bible))

    return pairs


print(create_dataset(ogbible[:100000]))

IndexError: list index out of range

Für dummies:
(target word, context word) in tensor and selbem index.

Wir feeden einen onehot vektor für das target word und createn einen embedded vector (verändert von weights layer one).  Embedded vector ist deutlich kleiner als der input.
Der kommt in einen output layer. In dem wird softmax activation auf den embedded vector angewandt.

Für loss vergleichen wir output mit onehot vector von target mit cross-entropy.



negative sampling:
Turns skip-gram into binary classification task by having real (pos) and fake(negative) examples. 
The objective of the network now is to distinct them.

negative examples have the the same target-word as the example before but a random selected context

In [ ]:
# Preprocess to return pairs onehot encoded



# Model

In [ ]:
class Embedder(Layer):
    def __init__(self, vocab_size):
        super(Embedder, self).__init__()
        self.embedder = Embedding(vocab_size, 64)
        self.classifier = Embedding(64, vocab_size)
        self.activation = Softmax()


    @tf.function
    def call(self, input):
        x = self.embedder(input)
        x = self.classifier(x)
        x = self.activation(x)

        return x

# Training

In [ ]:
# compute the loss of an input for the model and optimize/tweak according the parameters
def train_step(model, input, target, loss_function, optimizer):
    # use tf.gradientTape to compute loss, then gradients and apply these to the model to modify the parameters
    with tf.GradientTape() as tape:
        prediction = model(input)
        loss = loss_function(target, prediction)
        gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss


# compute the differences between or model prediction and the label, -> Supervision
def test(model, test_data, loss_function):
  # test over complete test data
  test_loss_aggregator = []
  for (input, target) in test_data:

    prediction = model(input)
    
    sample_test_loss = loss_function(target, prediction)
    
    test_loss_aggregator.append(sample_test_loss.numpy())
    
# for all input and computed losses get the mean of accuracy and loss and return them
  test_loss = tf.reduce_mean(test_loss_aggregator)

  return test_loss

maybe categorical cross entropy
training data 

In [ ]:
#predefine learning-rate and epochs
num_epochs = 10
alpha = 0.001

# create a model
model = Embedder()
model.summary
# define loss-function and optimizer
cross_entropy_loss = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# create empty arrays to store test/accuracy values, to track the network progress
train_losses = []
test_losses = []

# get initial accuracy- and loss valus before training starts
test_loss= test(model, test_ds, cross_entropy_loss)
test_losses.append(test_loss)

train_loss= test(model, train_ds, cross_entropy_loss)
train_losses.append(train_loss)

print("Starting Training AutoEncoder ")
# training loop
average_time = []
for epoch in range(num_epochs):
    # print accuracy of each epoch
    pre_train_time = time.time()
    
    loss_epoch = []
    # for all input, do a forwardstep and obtain loss
    for input, target in tqdm(train_ds_noisy):
        train_loss = train_step(model, input, target, cross_entropy_loss, optimizer)
        loss_epoch.append(train_loss)
    # get the mean loss of this epoch by using reduce_sum of TF over all input-losses and appending to the array  
    train_losses.append(tf.reduce_mean(loss_epoch))
    
    # get the losses and accuracy of this epoch and store them
    test_loss = test(model, test_ds_noisy, cross_entropy_loss)
    test_losses.append(test_loss)

    average_time.append(time.time() - pre_train_time)
    print("Took: " + str(time.time() - pre_train_time))
    print("Loss for this epoch: " + str(test_loss))
    
# print accuracy after 10 epochs
print("Mean Time per Epoch: " + str(round(np.mean(average_time), 2)) + "! With Parameters: ")

#print("Batch size: " + str(BATCH_SIZE) + "  Prefetch size: " + str(PREFETCH_SIZE) + "  Buffer size: " + str(BUFFER_SIZE))



 nearest neighbours according to the(cosine similarity) of each epoch

 1. Calculate the cosine similarities between the whole embedding and the
embedding of the words you want to investigate
2. For each selected word, sort the neighbours by their distance and return
the k-nearest ones.